In [1]:
from onc import ONC
import os
import sys
import pandas as pd
import datetime

In [2]:
# Get the current script's directory
current_dir = os.path.dirname(os.path.abspath('/Users/schlesin/Library/CloudStorage/GoogleDrive-schlesin@oceannetworks.ca/My Drive/ONC Projects/Activefolder_Mac/DailyChecks/*'))# Get the parent directory by going one level up

sys.path.append(current_dir)
print(current_dir)
from getToken import Token

onc = ONC(token=Token())

/Users/schlesin/Library/CloudStorage/GoogleDrive-schlesin@oceannetworks.ca/My Drive/ONC Projects/Activefolder_Mac/DailyChecks
/Users/schlesin/Library/Application Support/ONC-Token
Token file exists.


In [3]:
def getDeviceIds(deviceCode):
    import psycopg2
    url = 'select deviceid from device where devicecode = '+"'"+deviceCode+"'"
    conn = psycopg2.connect("dbname='dmas' user='www' host='142.104.193.110' password='poseidon'")
    cur = conn.cursor()
    cur.execute(url)
    n1 = (cur.fetchall())
    return(n1)

    

In [4]:
def getLastAnnotation(deviceCode):
    import requests
    deviceId = str(getDeviceIds(deviceCode)[0][0])

    #  query to not include topology
    url = 'https://data.oceannetworks.ca/AnnotationServiceV3?annotationSources=3%2C5&resourceTypeId=2&includeTopology=true&resourceId='+deviceId+'&fields=%7B%7D&levelIndexes=0%2C1%2C2%2C3%2C4%2C5&pageSize=1000&pageNum=1'
    r = requests.get(url)

    df = pd.DataFrame(r.json()['payload']['naiveAnnotationList'])
    return(df)

In [50]:
df = pd.DataFrame()

In [51]:
deviceCategoryCodes =['BBS','SPS','Accelerometer','Tiltmtr']
dateFrom = ((datetime.datetime.now()-datetime.timedelta(10))).strftime("%Y-%m-%dT%H:%M:%S")+'.000Z'
dateTo = (datetime.datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")+'.000Z'
print(dateFrom)
print(dateTo)
df
for dC in deviceCategoryCodes:
    data = onc.getDeployments({'deviceCategoryCode':dC,
                               'dateFrom':dateFrom,
                               'dateTo':dateTo})
    df2 = pd.DataFrame(data)
    df2['annotation']=None
    for i,j in enumerate(df2['deviceCode'].values):
        try:
            annot = getLastAnnotation(j).sort_values('startDate',ascending=False).iloc[0]
            #print(dict(annot))
        except:
            annot=dict()
        df2['annotation'][i]=dict(annot)
    df = pd.concat([df,df2],ignore_index=True)

2024-09-28T11:59:21.000Z
2024-10-08T11:59:21.000Z


/var/folders/l3/qcsngsps05340xvrbf2kk0hrcfxl6n/T/ipykernel_46606/3342825669.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['annotation'][i]=dict(annot)
/var/folders/l3/qcsngsps05340xvrbf2kk0hrcfxl6n/T/ipykernel_46606/3342825669.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['annotation'][i]=dict(annot)
/var/folders/l3/qcsngsps05340xvrbf2kk0hrcfxl6n/T/ipykernel_46606/3342825669.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [52]:
df

,begin,citation,depth,deviceCategoryCode,deviceCode,end,hasDeviceData,heading,lat,locationCode,lon,pitch,roll,annotation
0,2022-07-31T21:54:54.000Z,None,2321.0,BBS,MARISSEISMOMETERTR1302,None,True,344.0,47.958350,ENEF,-129.035530,None,None,"{'annotationId': 11113570, 'annotationSource':..."
1,2016-06-21T17:35:32.000Z,{'citation': 'Ocean Networks Canada Society. 2...,2361.0,BBS,GURALPBBSEISMO2533,None,True,0.0,47.959766,ENWF,-129.124479,None,None,"{'annotationId': 11314950, 'annotationSource':..."
2,2023-06-29T10:41:14.000Z,None,2276.0,BBS,GURALPORCUSBBST310920,None,True,88.0,47.924023,KEMO,-129.108121,None,None,"{'annotationId': 11946800, 'annotationSource':..."
3,2023-07-12T02:34:50.000Z,None,2656.0,BBS,GURALPORCUSBBST311164,None,True,360.0,47.762580,NC27,-127.758102,None,None,"{'annotationId': 15332500, 'annotationSource':..."
4,2009-09-17T13:18:56.000Z,{'citation': 'Ocean Networks Canada Society. 2...,1256.0,BBS,GURALPBBSEISMO2217,None,True,12.0,48.670537,NC89,-126.848767,None,None,"{'annotationId': 15242000, 'annotationSource':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2018-06-24T08:34:57.000Z,{'citation': 'Ocean Networks Canada Society. 2...,2656.0,TILTMTR,RBRTILTMETERACCBPR63057,None,True,310.0,47.756717,CBC27.Z1,-127.731602,None,None,"{'annotationId': 13549800, 'annotationSource':..."
70,2022-08-05T06:10:45.000Z,None,1748.0,TILTMTR,RBRQUARTZ3APT207597,None,True,0.0,48.618407,CDFE,-127.038060,None,None,{}
71,2021-08-08T09:34:32.000Z,None,2504.0,TILTMTR,RBRQUARTZ3APT207598,None,True,108.0,48.599443,CDFM,-127.079427,None,None,{}
72,2021-08-09T04:13:13.000Z,None,2559.0,TILTMTR,RBRQUARTZ3APT207599,None,True,0.0,48.570747,CDFW,-127.137591,None,None,{}


In [54]:
df3 = df.set_index(df['deviceCode'])

In [55]:
df3.drop(columns='deviceCode',inplace=True)

In [66]:
#df.drop(columns=['begin'],inplace=True)
df3.to_json('SeismicInstruments.json',orient='index',indent=4)